In [24]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from pymystem3 import Mystem
import re
import os
import numpy as np
import pandas as pd
import csv
from keras.models import Sequential, load_model, Model
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Embedding, Input, Concatenate
from keras.layers.normalization import BatchNormalization
from keras import applications
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import LSTM, Concatenate, Dense, Input, Embedding, Bidirectional, AlphaDropout, Masking, GRU
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [22]:
def read_and_split(filename):
    df=pd.read_csv(filename).fillna("")
    train,test=train_test_split(df,test_size=0.2)
    return train,test

def clean(x):
    return ''.join(m.lemmatize(re.sub('([^а-яa-z]+)',' ',x.lower()))).strip()

def prepare_for_rnn(train,test):
    train.comment=train.comment.apply(clean)
    train.commentNegative=train.commentNegative.apply(clean)
    train.commentPositive=train.commentPositive.apply(clean)

    test.comment=test.comment.apply(clean)
    test.commentNegative=test.comment.apply(clean)
    test.commentPositive=test.comment.apply(clean)
    
    tkn=Tokenizer(filters="")
    tkn.fit_on_texts(train.comment+train.commentNegative+train.commentPositive)
    
    comments=tkn.texts_to_sequences(train.comment)
    comments_neg=tkn.texts_to_sequences(train.commentNegative)
    comments_pos=tkn.texts_to_sequences(train.commentPositive)

    t_comments=tkn.texts_to_sequences(test.comment)
    t_comments_neg=tkn.texts_to_sequences(test.commentNegative)
    t_comments_pos=tkn.texts_to_sequences(test.commentPositive)

    c_len=int(np.percentile(list(map(len,comments)),95))
    cneg_len=int(np.percentile(list(map(len,comments_neg)),95))
    cpos_len=int(np.percentile(list(map(len,comments_pos)),95))

    c_pad=pad_sequences(comments,c_len)
    cneg_pad=pad_sequences(comments_neg,cneg_len)
    cpos_pad=pad_sequences(comments_pos,cpos_len)

    t_c_pad=pad_sequences(t_comments,c_len)
    t_cneg_pad=pad_sequences(t_comments_neg,cneg_len)
    t_cpos_pad=pad_sequences(t_comments_pos,cpos_len)

    y=((train.reting.values.astype(np.float32))-1)/4

    return c_pad,cneg_pad,cpos_pad,t_c_pad,t_cneg_pad,t_cpos_pad,y

def prepare_for_cnn(train,test):
    words = 256
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train.comment+train.commentNegative+train.commentPositive)
    sequences = tokenizer.texts_to_sequences(train.comment)
    word_index = tokenizer.word_index
    X_train = pad_sequences(sequences, maxlen = words)

    pos_sequences = tokenizer.texts_to_sequences(train.commentPositive)
    pX_train = pad_sequences(pos_sequences, maxlen = words)

    neg_sequences = tokenizer.texts_to_sequences(train.commentNegative)
    nX_train = pad_sequences(neg_sequences, maxlen = words)

    s = tokenizer.texts_to_sequences(test.comment)
    X_test = pad_sequences(s, maxlen=words)

    s = tokenizer.texts_to_sequences(test.commentPositive)
    pX_test = pad_sequences(s, maxlen=words)

    s = tokenizer.texts_to_sequences(test.commentNegative)
    nX_test = pad_sequences(s, maxlen=words)
    
    y_train=((train.reting.values.astype(np.float32))-1)/4
    return X_train,pX_train,nX_train,X_test,pX_test,nX_test,y_train

def train_rnn(c_pad,cneg_pad,cpos_pad,y):
    n_words=23023
    comm=Input((c_pad.shape[1],))
    cneg=Input((cneg_pad.shape[1],))
    cpos=Input((cpos_pad.shape[1],))

    m_comm=Masking()(comm)
    m_cneg=Masking()(cneg)
    m_cpos=Masking()(cpos)

    enc_lstm=Bidirectional(LSTM(256,return_sequences=True,dropout=0.2))
    enc2_lstm=LSTM(256,dropout=0.2)

    emb=Embedding(n_words,64)

    comm_emb=emb(m_comm)
    cneg_emb=emb(m_cneg)
    cpos_emb=emb(m_cpos)

    comm_enc=enc_lstm(comm_emb)
    cneg_enc=enc_lstm(cneg_emb)
    cpos_enc=enc_lstm(cpos_emb)

    comm_enc2=enc2_lstm(comm_enc)
    cneg_enc2=enc2_lstm(cneg_enc)
    cpos_enc2=enc2_lstm(cpos_enc)

    conc=Concatenate()([comm_enc2,cneg_enc2,cpos_enc2])

    res=Dense(64,activation="selu")(conc)
    res=Dense(1,activation="sigmoid")(res)

    model=Model([comm,cneg,cpos],res)
    model.compile("adam","mse")
    model.fit([c_pad,cneg_pad,cpos_pad],y,batch_size=512,epochs=3)
    return model
    
def train_cnn(X_train,pX_train,nX_train,y_train):
    n_words=23109
    words = 256
    
    embedding_layer = Embedding(n_words, 50, input_length=words)

    sequence_input = Input(shape=(words, ), dtype='float32')
    embedded_sequences = embedding_layer(sequence_input)

    pos_sequence_input = Input(shape=(words, ), dtype='float32')
    pos_embedded_sequences = embedding_layer(pos_sequence_input)

    neg_sequence_input = Input(shape=(words, ), dtype='float32')
    neg_embedded_sequences = embedding_layer(neg_sequence_input)

    x = Concatenate()([embedded_sequences, pos_embedded_sequences, neg_embedded_sequences])
    x = Dropout(0.9)(x)
    x = Conv1D(256, 4, activation='relu', padding='same')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(256, 4, activation='relu', padding='same')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(256, 4, activation='relu', padding='same')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(256, 4, activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dropout(0.9)(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[sequence_input, pos_sequence_input, neg_sequence_input], outputs=x)
    model.compile(loss='mse', optimizer='adam')
    model.fit([X_train, pX_train, nX_train], y_train, epochs=10, batch_size=512)
    return model

In [23]:
print("Reading data:")
train,test=read_and_split("X_train.csv")
print("Preparing data for RNN:")
m=Mystem()
c_pad,cneg_pad,cpos_pad,t_c_pad,t_cneg_pad,t_cpos_pad,y=prepare_for_rnn(train,test)
print("Preparing data for CNN:")
X_train,pX_train,nX_train,X_test,pX_test,nX_test,y_train=prepare_for_cnn(train,test)
print("Training RNN:")
rnn=train_rnn(c_pad,cneg_pad,cpos_pad,y)
print("Training CNN:")
cnn=train_cnn(X_train,pX_train,nX_train,y_train)

Reading data:
Preparing data for RNN:
Preparing data for CNN:
Training RNN:
Epoch 1/3
12469/12469 [==============================] - 14s - loss: 0.1178    
Epoch 2/3
12469/12469 [==============================] - 12s - loss: 0.0720    
Epoch 3/3
12469/12469 [==============================] - 12s - loss: 0.0429    
Training CNN:
Epoch 1/10
12469/12469 [==============================] - 5s - loss: 0.1232     
Epoch 2/10
12469/12469 [==============================] - 2s - loss: 0.1123     
Epoch 3/10
12469/12469 [==============================] - 2s - loss: 0.1091     
Epoch 4/10
12469/12469 [==============================] - 2s - loss: 0.0928     
Epoch 5/10
12469/12469 [==============================] - 2s - loss: 0.0821     
Epoch 6/10
12469/12469 [==============================] - 2s - loss: 0.0753     
Epoch 7/10
12469/12469 [==============================] - 2s - loss: 0.0720     
Epoch 8/10
12469/12469 [==============================] - 2s - loss: 0.0664     
Epoch 9/10
12469/12469

In [25]:
pred_rnn=rnn.predict([t_c_pad,t_cneg_pad,t_cpos_pad])*4+1
pred_cnn=cnn.predict([X_test,pX_test,nX_test])*4+1

In [38]:
final_pred=np.mean([pred_rnn,pred_cnn],axis=0)
print("Mean squared error: {}\nMean absolute error: {}\nR^2 score: {}".format(mean_squared_error(test.reting,pred_rnn),
                                                                          mean_absolute_error(test.reting,final_pred),
                                                                          r2_score(test.reting,final_pred)))

Mean squared error: 1.0420059276390916
Mean absolute error: 0.9345256601640092
R^2 score: 0.21161647416071705
